# Data Formatting

### import modules

In [51]:
import csv
import xml.etree.ElementTree as ET
import re
from numpy import array
import cpplotter
from cpcsc import get_node_count
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from statistics import mean
from numpy import nan

### Variables declaration

In [88]:
nodes_with_eh=[2,3,4,5,6]
battery_capacity=265420800
period = 30
cutoff = 16800
wrap = True
panel_area = 100 # area of the solar panel, in cm²
factor=panel_area*32768/1000000      # 32768 ticks per second     #/1000000 uW to W

## 1- Irradiation data formatting

### Sampling reduction of original dataset

In [53]:
mylines = []
dataset_sampling_period=0.5    #in minutes
target_sampling_period=30      #in minutes
with open('columbia_irr_only_no_gaps.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for myline in spamreader:
        x=myline[0].split(",")
        mylines.append(float(x[1]))
s=int(target_sampling_period/dataset_sampling_period)
i=0
j=0
while(1):
    try:
#         print("%d" %j +",%.2f" % (mylines[i]*s))
        i+=s
        j+=1
        if j==560:
            break
    except:
        exit()

In [54]:
irr=pd.read_csv("dataset.csv",index_col="id")

## 2- Log listener data formatting

In [175]:
def log_parse(target):
    data=[]
    with open(target, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in spamreader:
            data.append(row)

    data=pd.DataFrame(data)
    data.columns=["time", "ID", "data"]
    time=[]
    id=[]
    for i in data["time"]:
        l=len(i)          #length of time str
        if l==9:             # if time format:   55:20.325
            time.append(int(i[0:2]))                                    # time in minutes
        else:                # if time format: 1:55:20.325
            limit=i.find(':') 
            time.append(int(i[0:limit])*60+int(i[limit+1:limit+3]))   # time in minutes
    for i in data["ID"]:
        id.append(int(i[i.find(':')+1:]))

    cpu=[]
    lpm=[]
    tx=[]
    rx=[]
    rem_energy=[]

    for i in data["data"]:
        if not i.find('[BATT] Ticks diff'):
            lpm.append(int(i[i.find('LPM')+4:i.find('TX')-2]))
            cpu.append(int(i[23:i.find('LPM')-2]))
            rx.append(int(i[i.find('RX')+3:]))
            tx.append(int(i[i.find('TX')+3:i.find('RX')-2]))
            rem_energy.append(None)
        elif not i.find('[BATT] Energy'):                #find remaining energy lines 
            rem_energy.append(int(i[25:i.find('.')]))
            cpu.append(None)
            lpm.append(None)
            rx.append(None)
            tx.append(None)
        else:
            rem_energy.append(None)
            cpu.append(None)
            lpm.append(None)
            rx.append(None)
            tx.append(None)

    time_df=pd.DataFrame({'time': time})
    id_df=pd.DataFrame({'ID': id})
    data.update(time_df)
    data.update(id_df)
    data.insert(2,'remaining energy',rem_energy,True)
    data.insert(3,'lpm',lpm,True)
    data.insert(4,'cpu',cpu,True)
    data.insert(5,'rx',rx,True)
    data.insert(6,'tx',tx,True)
    data.drop('data', axis=1, inplace=True)

      
    irr['harv'] = round(irr['harv'].multiply(factor),2)    # in W*tick during 30 minutes

    harv = pd.DataFrame(columns=['time','ID','harvested energy'])
    
    for i in nodes_with_eh:
        harvested=data.loc[data['ID'] == i][['time','ID',"remaining energy"]].dropna().reset_index()
        harvested.insert(2, "harvested energy", irr, True) 
        harvested=harvested.set_index('index')
        harv=pd.concat([harv,harvested])

    harv=harv.drop(["time","ID","remaining energy"],axis=1)
    data=pd.concat([data,harv], axis=1)
    
    
    df = pd.DataFrame(columns=["remaining energy after eh", "added"])
    for n in nodes_with_eh:    
        list_after_eh=[]
        l=data.loc[data['ID'] == n][['remaining energy','harvested energy']].dropna()
        index=l.index.values.tolist()
        list=l.values.tolist()
        add=0
        for i in list:
            if i[0]+i[1]+add>battery_capacity:
                i[0]=battery_capacity
                i[1]=battery_capacity-i[0]
                add+=battery_capacity-i[0]
            else:
                add+=i[1]
                i[0]+=add
            list_after_eh.append(i)
        add=0
        df = pd.concat([df,pd.DataFrame(list_after_eh, columns=["remaining energy after eh", "added"], index=index)])

    data=pd.concat([data,df], axis=1)
    
    
    return(data)

In [176]:
data=log_parse("loglistener.txt")
data.to_csv('data.csv', index=False)

In [177]:
data=pd.read_csv("data.csv")